# Урок 4. Парсинг HTML. XPath

In [1]:
from lxml import html
import requests
from datetime import datetime, timedelta

In [2]:
def news_lenta_ru():
    
    news = []
    
    keys = ('title', 'date', 'link')
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    link_lenta = 'https://lenta.ru/'
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36 OPR/70.0.3728.133 (Edition Yx)'
    }
    
    request = requests.get(link_lenta, headers = headers)

    root = html.fromstring(request.text)
    root.make_links_absolute(link_lenta)
    
    news_links = root.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                                //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                                /a/@href''')
    
    news_text = root.xpath('''(//section[@class="row b-top7-for-main js-top-seven"]//div[@class="first-item"]/h2 | 
                                //section[@class="row b-top7-for-main js-top-seven"]//div[@class="item"])
                                /a/text()''')
    
    for i in range(len(news_text)):
        news_text[i] = news_text[i].replace(u'\xa0', u' ')
    
    news_date = []
    
    for item in news_links:
        request = requests.get(item)
        root = html.fromstring(request.text)
        date = root.xpath("//div[@class='b-topic__info']/time/@datetime")
        news_date.extend(date)

    for i in range(len(news_date)):
        news_date[i] = datetime.strptime(news_date[i], date_format)
        
    for item in list(zip(news_text, news_date, news_links)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        
        news_dict['source'] = 'lenta.ru'
        news.append(news_dict)
    
    return news

In [3]:
def news_mail_ru():
    
    news = []    
    
    headers = {
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
                }
    
    keys = ('title', 'date', 'link', 'source')
    date_format = '%Y-%m-%dT%H:%M:%S%z'
    

    link_mail_ru = 'https://news.mail.ru/politics/'
    
    request = requests.get(link_mail_ru, headers=headers)
    root = html.fromstring(request.text)
    
    news_links = root.xpath("//div[@class ='paging js-module js-module']//a[@class = 'newsitem__title link-holder']/@href")
    
    news_text = root.xpath("//div[@class ='paging js-module js-module']//a[@class = 'newsitem__title link-holder']/span/text()")
    
    for i in range(len(news_text)):
        news_text[i] = news_text[i].replace(u'\xa0', u' ')
      
    news_date = root.xpath("//span[@class = 'newsitem__param js-ago']/@datetime")
     
    for i in range(len(news_date)):
        news_date[i] = datetime.strptime(news_date[i], date_format)
        
    news_source =  root.xpath("//span[@class = 'newsitem__param']/text()")
        
    for item in list(zip(news_text, news_date, news_links, news_source)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        news.append(news_dict)
    
    return news

In [4]:
def news_yandex_ru():
    
    news = []    
    
    headers = {
                   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'
                }

    date_format = '%Y-%m-%dT%H:%M:%S%z'
    
    keys = ('title', 'date', 'link', 'source')

    
    link_yandex_ru = 'https://yandex.ru/news/region/tula'
    
    request = requests.get(link_yandex_ru, headers=headers)
    root = html.fromstring(request.text)
    
    news_links_1 = root.xpath("//div[@class = 'mg-grid__row mg-grid__row_gap_8 news-top-stories news-app__top']//a[@class ='news-card__link']/@href")

    news_links_2 = root.xpath("//div[@class = 'mg-grid__row mg-grid__row_gap_8 news-feed news-app__feed']//a[@class ='news-card__link']/@href")

    news_links = news_links_1 + news_links_2

    news_text_1 = root.xpath("//div[@class = 'mg-grid__row mg-grid__row_gap_8 news-top-stories news-app__top']//a[@class ='news-card__link']/h2/text()")

    news_text_2 = root.xpath("//div[@class = 'mg-grid__row mg-grid__row_gap_8 news-feed news-app__feed']//a[@class ='news-card__link']/h2/text()")

    news_text = news_text_1 + news_text_2

    news_date = root.xpath("//span[@class = 'mg-card-source__time']/text()")

    for i in range(len(news_date)):
        if 'вчера' in news_date[i]:
            news_date[i] = news_date[i].split()[2]
            news_date[i] = (datetime.today()-timedelta(days=1)).strftime("%Y-%m-%d")+'T'+news_date[i]+':00+03:00'
        else:
            news_date[i] = datetime.today().strftime("%Y-%m-%d")+'T'+news_date[i]+':00+03:00'
        news_date[i] = datetime.strptime(news_date[i], date_format)
    
    news_source =  root.xpath("//span[@class = 'mg-card-source__source']/a/text()")
        
    for item in list(zip(news_text, news_date, news_links, news_source)):
        news_dict = {}
        for key, value in zip(keys, item):
            news_dict[key] = value
        news.append(news_dict)
    
    return news

In [5]:
summ_dict = news_yandex_ru() + news_mail_ru() + news_lenta_ru()

In [6]:
summ_dict

[{'title': 'В Тульской области около 28000 человек прошли вакцинацию от гриппа',
  'date': datetime.datetime(2020, 9, 9, 15, 45, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))),
  'link': 'https://yandex.ru/news/story/V_Tulskoj_oblasti_okolo_28000_chelovek_proshli_vakcinaciyu_ot_grippa--9d5c4fa47b72c25ebedafd25a83c525d?lang=ru&rubric=Tula&stid=U6okHxi1nLCeawk2pcPR&t=1599670937&persistent_id=112046444',
  'source': 'Тульская пресса'},
 {'title': 'Депутат гордумы Владимир Тимаков прокомментировал решение ЕСПЧ',
  'date': datetime.datetime(2020, 9, 9, 18, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))),
  'link': 'https://yandex.ru/news/story/Deputat_gordumy_Vladimir_Timakov_prokommentiroval_reshenie_ESPCH--4588b38e46a4073095aaf011a09b59e7?lang=ru&rubric=Tula&stid=uqFpSoREw1ZBGFekBUOj&t=1599670937&persistent_id=111922204',
  'source': 'Тульская пресса'},
 {'title': 'В Пролетарском округе Тулы снова отключат воду из-за ремонтных работ',
  'date': datetime.date

In [7]:
from pymongo import MongoClient

In [8]:
def add_in_mongodb(dictionary):
    mongodb = MongoClient('localhost', 27017)
    db = mongodb['news_db']
    collection = db['news']
    
    for news in dictionary:
         db.collection.insert_one(news)
        
    return "Done"

In [9]:
add_in_mongodb(summ_dict)

'Done'